# CHECK WHICH GEMINI MODELS ARE AVAILABLE

# Install/upgrade packages 
!pip install --upgrade google-generativeai langchain-google-genai

In [ ]:
# Check what's available
import google.generativeai as genai
import os
from getpass import getpass

# Get API key
if "GOOGLE_API_KEY" not in os.environ:
    api_key = getpass("Enter your Google API Key: ")
    os.environ["GOOGLE_API_KEY"] = api_key

# Configure and list models
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

print("=" * 70)
print("Available Gemini models:")
print("=" * 70)

available_models = []
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        model_name = model.name.replace('models/', '')
        available_models.append(model_name)
        print(f"✅ {model_name}")

print("\n" + "=" * 70)
print(f"Found {len(available_models)} available models")
print("=" * 70)

# GOAL ORIENTED PROMPT APPROACH - SUCCESS AND FAILURE CASES

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from getpass import getpass
from IPython.display import display, Markdown

from medical_appointment_agent import (
    get_rate_limit_safe_llm,
    create_task_oriented_agent,
    create_goal_oriented_agent,
    run_task_oriented_workflow,
    create_llm_with_fallback,
    RateLimitError,
)

import time

/Users/rajeevkulkarni/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/rajeevkulkarni/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rajeevkulkarni/Library/Python/3.9/lib/python/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade

In [3]:
# Setup available (rate limit safe) LLM from Gemini
api_key = getpass("Enter your Google API Key: ")
os.environ["GOOGLE_API_KEY"] = api_key

llm = get_rate_limit_safe_llm(verbose=True)

🤖 Initializing LLM with model: gemini-2.5-flash
✅ Successfully initialized with gemini-2.5-flash


In [4]:
goal_agent = create_goal_oriented_agent(llm)
# Single goal request
response = goal_agent.invoke({
    "input": "Book me an appointment with a cardiologist"
})
print("\n📋 FINAL RESULT:")
display(Markdown(response["output"]))



> Entering new AgentExecutor chain...
Action: FindDoctorsTool
Action Input: CardiologyAvailable doctors in Cardiology: Dr. Patel, Dr. Lee, Dr. Martinez, Dr. JohnsonAction: CheckAvailabilityTool
Action Input: Dr. PatelAvailable slots for Dr. Patel: Jan 21 9AM, Jan 22 2PM, Jan 24 10AM, Jan 25 3PMAction: BookAppointmentTool
Action Input: Dr. Patel for Jan 21 9AM✅ Appointment booked: Dr. Patel for Jan 21 9AMgoal is complete
Final Answer: Your appointment with Dr. Patel has been booked for Jan 21 9AM.

> Finished chain.

📋 FINAL RESULT:


Your appointment with Dr. Patel has been booked for Jan 21 9AM.

# TASK ORIENTED PROMPT APPROACH - SUCCESS AND FAILURE CASES 

In [ ]:
# Setup
# api_key = getpass("Enter your Google API Key: ")
# os.environ["GOOGLE_API_KEY"] = api_key

In [5]:
## Success Case # 1
# Try with automatic model fallback
try:
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "cardiologists")
    print(result)
    
except RateLimitError as e:
    print(f"\n⚠️  {e}")
    print("💤 Waiting 60 seconds before retrying...")
    time.sleep(60)
    
    # Retry
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "cardiologists")
    print(result)

🤖 Initializing LLM with model: gemini-2.5-flash
✅ Successfully initialized with gemini-2.5-flash
WORKFLOW: Find cardiologists → Check Availability → Book Appointment

STEP 1: Find cardiologists


> Entering new AgentExecutor chain...
Action: FindDoctorsTool
Action Input: CardiologyAvailable doctors in Cardiology: Dr. Patel, Dr. Lee, Dr. Martinez, Dr. JohnsonI have the answer
Final Answer: Available doctors in Cardiology: Dr. Patel, Dr. Lee, Dr. Martinez, Dr. Johnson

> Finished chain.

📋 OUTPUT: Available doctors in Cardiology: Dr. Patel, Dr. Lee, Dr. Martinez, Dr. Johnson

✅ Found 4 doctor(s): Dr. Johnson, Dr. Martinez, Dr. Patel, Dr. Lee

STEP 2: Check availability for Dr. Johnson


> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 58.294755746s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 5
}
, retry_

Action: CheckAvailabilityTool
Action Input: Dr. JohnsonAvailable slots for Dr. Johnson: Jan 20 2PM, Jan 22 10AM, Jan 25 1PM, Jan 27 11AMI have the answer
Final Answer: Available slots for Dr. Johnson: Jan 20 2PM, Jan 22 10AM, Jan 25 1PM, Jan 27 11AM

> Finished chain.

📋 OUTPUT: Available slots for Dr. Johnson: Jan 20 2PM, Jan 22 10AM, Jan 25 1PM, Jan 27 11AM

✅ Found available slot: Jan 27 11AM

STEP 3: Book appointment with Dr. Johnson at Jan 27 11AM


> Entering new AgentExecutor chain...
Action: BookAppointmentTool
Action Input: Dr. Johnson for Jan 27 11AM✅ Appointment booked: Dr. Johnson for Jan 27 11AMFinal Answer: ✅ Appointment booked: Dr. Johnson for Jan 27 11AM

> Finished chain.

📋 OUTPUT: ✅ Appointment booked: Dr. Johnson for Jan 27 11AM

✅ WORKFLOW COMPLETED SUCCESSFULLY!
   Doctor: Dr. Johnson
   Specialty: cardiologists
   Time: Jan 27 11AM
{'success': True, 'specialty': 'cardiologists', 'doctor': 'Dr. Johnson', 'time_slot': 'Jan 27 11AM', 'error': None}


In [ ]:
## Success Case # 2
result = run_task_oriented_workflow(agent, "heart specialists")
display(Markdown(result))

In [6]:
## Failure case # 1
# Try with automatic model fallback
try:
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "dermatologist")  # Not available
    
except RateLimitError as e:
    print(f"\n⚠️  {e}")
    print("💤 Waiting 60 seconds before retrying...")
    time.sleep(60)
    
    # Retry
    llm = create_llm_with_fallback(verbose=True)
    agent = create_task_oriented_agent(llm)
    result = run_task_oriented_workflow(agent, "dermatologist")  # Not available

display(Markdown(result['output']))

🤖 Initializing LLM with model: gemini-2.5-flash
✅ Successfully initialized with gemini-2.5-flash
WORKFLOW: Find dermatologist → Check Availability → Book Appointment

STEP 1: Find dermatologist


> Entering new AgentExecutor chain...
Action: FindDoctorsTool
Action Input: dermatologistERROR: No doctors found for specialty: DermatologyI have the answer
Final Answer: ERROR: No doctors found for specialty: Dermatology

> Finished chain.

📋 OUTPUT: ERROR: No doctors found for specialty: Dermatology

❌ WORKFLOW STOPPED: No doctors found. Steps 2 and 3 will not execute.


KeyError: 'output'

In [ ]:
## Failure case # 2
result = run_task_oriented_workflow(agent, "urologist")      # Not available
display(Markdown(result))